In [1]:
#imports
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
import re
import nltk
from sklearn import feature_extraction
import math
from features import word2vec_features, createWord2VecDict, sentiment_features
from features import word_overlap_features, refuting_features, polarity_features
from sklearn.ensemble import GradientBoostingClassifier
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from score import report_score, LABELS, score_submission
from deepModel import fnc_score, load_split_vectors, vectorsExist, implementDeepModel
import pickle

In [58]:
#Method

def generate_features(headline, body):
    X_overlap = word_overlap_features(headline, body)
    print('word overlap fets generated...')
    X_refuting = refuting_features(headline, body)
    print('refuting fets generated...')
    X_polarity = polarity_features(headline, body)
    print('polarity fets generated...')
    X = np.c_[X_polarity, X_refuting, X_overlap]
    return X

def preprocess(stances, bodies):
    processed_heads, processed_bodies = [], []
    
    # Cleans a string: Lowercasing, trimming, removing non-alphanumeric, stop words.
    for headline in stances:
        clean_head = " ".join(re.findall(r'\w+', headline, flags=re.UNICODE)).lower()
        tok_head = [t for t in nltk.word_tokenize(clean_head)]
        pro_head = [w for w in tok_head if w not in feature_extraction.text.ENGLISH_STOP_WORDS and len(w) > 1 and (len(w)!=2 or w[0]!=w[1]) ]
        processed_heads.append(' '.join(pro_head))
        
    for body in bodies:
        clean_body = " ".join(re.findall(r'\w+', body, flags=re.UNICODE)).lower()
        tok_body = [t for t in nltk.word_tokenize(clean_body)]
        pro_body = [w for w in tok_body if w not in feature_extraction.text.ENGLISH_STOP_WORDS and len(w) > 1 and (len(w)!=2 or w[0]!=w[1]) ]
        processed_bodies.append(' '.join(pro_body))
    
    return processed_heads, processed_bodies

def cross_validation_split(dataset, folds=3):
    dataset_split = []
    dataset_copy = dataset.tolist()
    fold_size = math.ceil(int(len(dataset) / folds))
    for i in range(folds):
        fold = []
        while len(fold) < fold_size:
            fold.append(dataset_copy.pop())
        dataset_split.append(fold)
    return dataset_split
    
def retrieveHeadBody(dataset_type, isSmall):
    l = os.getcwd().split('/')
    l.pop()
    l.pop()
    # if isSmall and dataset_type == 'train':
    #     file_head = '/'.join(l) + "/input_data/smallData.csv"
    # else:
    file_head = '/'.join(l) + "/input_data/"+dataset_type+"_stances.csv"    
    file_body = '/'.join(l) + "/input_data/"+dataset_type+"_bodies.csv"
    head = pd.read_csv(file_head)
    body = pd.read_csv(file_body)
    head_array = head.values
    body_array = body.values
    labels = head_array[:,2]
    stance_ids = head_array[:,1]
    body_ids = body_array[:,0]
    new_lab = []
    for i in labels:
        if i == 'unrelated':
            new_lab.append(3)
        if i == 'agree':
            new_lab.append(0)
        if i == 'discuss':
            new_lab.append(2)
        if i == 'disagree':
            new_lab.append(1)
    
    pHead, pBody = preprocess(head_array[:,0], body_array[:,1])
    return pHead, pBody, stance_ids, body_ids, new_lab

def prepare_train_data(isSmall):
    pHead, pBody, stance_ids, body_ids, new_lab = retrieveHeadBody('train', isSmall)
    trainHead, valHead, trainLab, valLab, idTrain, idVal = train_test_split(pHead, new_lab, stance_ids, test_size=0.20, random_state=42)

    
    valBody = []
    for fid in idVal:
        valBody.append(pBody[body_ids.tolist().index(fid)])
        
    trainBody = []
    for fid in idTrain:
        trainBody.append(pBody[body_ids.tolist().index(fid)])
    
    #tpHead, tpBody, tstance_ids, tbody_ids, tnew_lab = retrieveHeadBody('competition_test',isSmall)
    #createWord2VecDict(pHead, pBody, tpHead, tpBody)
    return trainHead, trainBody, trainLab, valHead, valBody, valLab

def prepare_test_data(isSmall):
    pHead, pBody, stance_ids, body_ids, new_lab = retrieveHeadBody('competition_test',isSmall)
    testBody = []
    for fid in stance_ids:
        testBody.append(pBody[body_ids.tolist().index(fid)])
    
    return pHead, testBody, new_lab

def print_label(res):
    if res == 3:
        print('agree')
    if res == 0:
        print('unrelated')
    if res == 2:
        print('discuss')
    if res == 1:
        print('disagree')

def execGradientBoosting(isSmall):
    # if vectorsExist() and not isSmall:
    #     trainSentiment_feats, valSentiment_feats, testSentiment_feats, train_wvFeats, val_wvFeats, test_wvFeats, trainBase_feats, valBase_feats, testBase_feats, trainLabels, valLabels, testLabels = load_split_vectors()
    # else:
        #tHeadLine, tBody, tLabels, vHeadLine, vBody, vLabels = prepare_data_folds()
        
    trainHeadLine, trainBody, trainLabels, valHeadLine, valBody, valLabels = prepare_train_data(isSmall)
    trainLabels = np.reshape(trainLabels,(len(trainLabels),1))
    valLabels = np.reshape(valLabels,(len(valLabels),1))
    
    print('Data prepared and loaded')
    trainSentiment_feats = sentiment_features(trainHeadLine, trainBody)
    print('Train sentiment features generated....')
    trainBase_feats = generate_features(trainHeadLine, trainBody)
    print('Train baseline features generated....')
    trainHead_wvfeats, trainBody_wvfeats = word2vec_features(trainHeadLine, trainBody)
    train_wvFeats = []
    for x in range(len(trainHead_wvfeats)):
        train_wvFeats.append(np.concatenate((trainHead_wvfeats[x], trainBody_wvfeats[x])))
    train_wvFeats = np.array(train_wvFeats)
    print('Train word2vec features generated....')
    
    
    valBase_feats = generate_features(valHeadLine, valBody)
    print('Validation baseline features generated....')
    valSentiment_feats = sentiment_features(valHeadLine, valBody)
    print('Validation sentiment features generated....')
    valHead_wvfeats, valBody_wvfeats = word2vec_features(valHeadLine, valBody)
    val_wvFeats = []
    for x in range(len(valHead_wvfeats)):
        val_wvFeats.append(np.concatenate((valHead_wvfeats[x], valBody_wvfeats[x])))
    val_wvFeats = np.array(val_wvFeats)
    print('Validation word2vec features generated....')
    
    testHeadLine, testBody, testLabels = prepare_test_data(isSmall)
    testHead_wvfeats, testBody_wvfeats = word2vec_features(testHeadLine, testBody)
    print('Test word2vec features generated....')
    testSentiment_feats = sentiment_features(testHeadLine, testBody)
    print('Test sentiment features generated....')
    testBase_feats = generate_features(testHeadLine, testBody)
    print('Test baseline features generated....')
    test_wvFeats = []
    for x in range(len(testHead_wvfeats)):
        test_wvFeats.append(np.concatenate((testHead_wvfeats[x], testBody_wvfeats[x])))
        
    
    
    train_X = np.hstack((train_wvFeats, trainSentiment_feats))
    train_X = np.hstack((train_X, trainBase_feats))
    val_X = np.hstack((val_wvFeats, valSentiment_feats))
    val_X = np.hstack((val_X, valBase_feats))
    
    test_X = np.hstack((test_wvFeats, testSentiment_feats))
    test_X = np.hstack((test_X, testBase_feats))
    
    clf = GradientBoostingClassifier(n_estimators=5, random_state=14128, verbose=True)
    clf.fit(train_X, trainLabels)
        
    test_pred = clf.predict_proba(test_X)
    test_p = np.argmax(test_pred, axis=1)
    testPredictions = [round(value) for value in test_p]
    val_pred = clf.predict_proba(val_X)
    val_p = np.argmax(val_pred, axis=1)
    valPredictions = [round(value) for value in val_p]
    
    
    accuracy = accuracy_score(valLabels, valPredictions)
    print("Accuracy over validation dataset: %.2f%%" % (accuracy * 100.0))
    print('Score over validation dataset set: ',fnc_score(valLabels, valPredictions))
    
    y_pred = pd.Series(valPredictions)
    valLabels = np.array(valLabels)
    valLabels = valLabels.reshape(valLabels.shape[0])
    y_true = pd.Series(valLabels)
    print('Confusion matrix over validation dataset: ')
    print(pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))
    
    
    accuracy = accuracy_score(testLabels, testPredictions)
    print("Accuracy over test dataset: %.2f%%" % (accuracy * 100.0))
    print('Score over test dataset set: ',fnc_score(testLabels, testPredictions))
    
    y_pred = pd.Series(testPredictions)
    testLabels = np.array(testLabels)
    testLabels = testLabels.reshape(testLabels.shape[0])
    y_true = pd.Series(testLabels)
    print('Confusion matrix over test data: ')
    print(pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))
    implementDeepModel(trainSentiment_feats, valSentiment_feats, testSentiment_feats, train_wvFeats, val_wvFeats, test_wvFeats, trainBase_feats, valBase_feats, testBase_feats, trainLabels, valLabels, testLabels)

In [3]:
execGradientBoosting(False)

Data prepared and loaded
Train sentiment features generated....
word overlap fets generated...
refuting fets generated...
polarity fets generated...
Train baseline features generated....
/home/sanket/IIITH/Sem 4/Project/Project/output_data/word2vec.pickle
Train word2vec features generated....
word overlap fets generated...
refuting fets generated...
polarity fets generated...
Validation baseline features generated....
Validation sentiment features generated....
/home/sanket/IIITH/Sem 4/Project/Project/output_data/word2vec.pickle
Validation word2vec features generated....
/home/sanket/IIITH/Sem 4/Project/Project/output_data/word2vec.pickle
Test word2vec features generated....
Test sentiment features generated....
word overlap fets generated...
refuting fets generated...
polarity fets generated...
Test baseline features generated....


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1       27238.4524            1.32m
         2       24250.3577           59.73s
         3       22055.5658           39.71s
         4       20337.5138           19.87s
         5       18956.3492            0.00s
Accuracy over validation dataset: 82.35%
Score over validation dataset set:  62.93865747177576
Confusion matrix over validation dataset: 
Predicted   1     2     3   All
True                           
0           3   182   518   703
1          15    45   120   180
2           0   937   842  1779
3           0    54  7279  7333
All        18  1218  8759  9995
Accuracy over test dataset: 75.96%
Score over test dataset set:  50.53749597682652
Confusion matrix over test data: 
Predicted    1     2      3    All
True                              
0            2   259   1642   1903
1            8    52    637    697
2            7  1302   3155   4464
3          266    90  17993  18349
All        283  1703  23427  25413
Train

## DEMO

In [5]:
#Preparing Train Data
isSmall = False
trainHeadLine, trainBody, trainLabels, valHeadLine, valBody, valLabels = prepare_train_data(isSmall)
trainLabels = np.reshape(trainLabels,(len(trainLabels),1))
valLabels = np.reshape(valLabels,(len(valLabels),1))
   
print('Data prepared and loaded')
trainSentiment_feats = sentiment_features(trainHeadLine, trainBody)
print('Train sentiment features generated....')
trainBase_feats = generate_features(trainHeadLine, trainBody)
print('Train baseline features generated....')
trainHead_wvfeats, trainBody_wvfeats = word2vec_features(trainHeadLine, trainBody)
train_wvFeats = []
for x in range(len(trainHead_wvfeats)):
    train_wvFeats.append(np.concatenate((trainHead_wvfeats[x], trainBody_wvfeats[x])))
train_wvFeats = np.array(train_wvFeats)
print('Train word2vec features generated....')
   
    
valBase_feats = generate_features(valHeadLine, valBody)
print('Validation baseline features generated....')
valSentiment_feats = sentiment_features(valHeadLine, valBody)
print('Validation sentiment features generated....')
valHead_wvfeats, valBody_wvfeats = word2vec_features(valHeadLine, valBody)
val_wvFeats = []
for x in range(len(valHead_wvfeats)):
    val_wvFeats.append(np.concatenate((valHead_wvfeats[x], valBody_wvfeats[x])))
val_wvFeats = np.array(val_wvFeats)
print('Validation word2vec features generated....')

Data prepared and loaded
Train sentiment features generated....
word overlap fets generated...
refuting fets generated...
polarity fets generated...
Train baseline features generated....
/home/sanket/IIITH/Sem 4/Project/Project/output_data/word2vec.pickle
Train word2vec features generated....
word overlap fets generated...
refuting fets generated...
polarity fets generated...
Validation baseline features generated....
Validation sentiment features generated....
/home/sanket/IIITH/Sem 4/Project/Project/output_data/word2vec.pickle
Validation word2vec features generated....


In [40]:
c = 0
for i in trainLabels:
    if i == 0:
        print(c)
        break
    c = c + 1    

8


In [42]:
print(trainLabels[8])
print(trainHeadLine[8])
print(trainBody[8])

[0]
video marine saved helmet taliban sniper attack
importance wearing helmet field battle perfectly illustrated video uploaded youtube marine video marines seen building search source gunfire near doorway single file bullet suddenly bounces kevlar helmet middle soldier men quickly retreat building inspect damage soldier helmet ascertaining marine unharmed regroup consider options afghanistan veteran sam arnold says video shot conducting joint helicopter raid zad district helmand province 2013 footage uploaded youtube month received just 200 000 views marines moments shot helmet videos telegraph men try happened man asked 200 women sleep terrifying headcam shows cyclist land feet hit car greatest speech sports coach trousers hands children baffled walkman cassette players danny macaskill releases breathtaking isle skye tricks video 10 amazing tips improve sleep


In [6]:
#Training
train_X = np.hstack((train_wvFeats, trainSentiment_feats))
train_X = np.hstack((train_X, trainBase_feats))
val_X = np.hstack((val_wvFeats, valSentiment_feats))
val_X = np.hstack((val_X, valBase_feats))
clf = GradientBoostingClassifier(n_estimators=5, random_state=14128, verbose=True)
clf.fit(train_X, trainLabels)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1       27238.4524            1.56m
         2       24250.3577            1.16m
         3       22055.5658           46.60s
         4       20337.5138           23.26s
         5       18956.3492            0.00s


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=5,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=14128, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=True,
                           warm_start=False)

In [52]:
#News In Indian Language
Language = 'hi'
HeadLine = 'लॉकडाउन में ऑड-ईवन की तरह खुलें बाजार, दिल्ली सरकार ने केंद्र को भेजा प्रस्ताव'
Body     = 'कोरोना वायरस संकट के बढ़ते मामलों के बीच दिल्ली सरकार की ओर से केंद्र सरकार को लॉकडाउन के लिए प्रस्ताव भेज दिया गया है. दिल्ली के स्वास्थ्य मंत्री सत्येंद्र जैन ने शुक्रवार कोरोना वायरस संकट को लेकर मीडिया से बात की. उन्होंने इस दौरान कहा कि दिल्ली सरकार की ओर से केंद्र सरकार को लॉकडाउन के लिए सुझाव भेजा जा चुका है. हम चाहते हैं कि बसों और मेट्रो को लिमिटेड तरीके से खोला जाए.सत्येंद्र जैन ने कहा कि हमने सरकार से पचास फीसदी तक मॉल खोलने को कहा है और ऑड-ईवन की तरह खुलें बाजार. इसके अलावा पब्लिक प्लेस में मास्क पहनना जरूरी कर देना चाहिए.'
Actual_Lable = 'agree'

In [53]:
from googletrans import Translator
translator = Translator()
srcLang = 'hi'
destLang   = 'en'
Eng_Headline = translator.translate(HeadLine, dest=destLang , src=srcLang).text
Eng_Body     = translator.translate(Body, dest=destLang , src=srcLang).text
print('Headline In English : ',Eng_Headline)
print('NewsBody In English : ',Eng_Body)

Headline In English :  Open market like the odd-even in lockdown, offer government sent center
NewsBody In English :  Corona virus from the central government on behalf of the government between the incidence of the crisis has sent the proposal to the lockdown. Health Minister Satyendra Jain Delhi on Friday to talk to the media the corona virus crisis. He has been sent suggestions during that lockdown central government on behalf of the government. We want buses and subway opened to limited ways Jaa.satyendra Jain said that we have asked to open the mall to the government to fifty percent and open market like the odd-even. Also should be required to wear masks in public place.


In [54]:
testHeadLine = [Eng_Headline]
testBody     = [Eng_Body]
testLabels   = [0]

In [55]:
#Preparing Test Data
#testHeadLine, testBody, testLabels = prepare_test_data(isSmall)
testHead_wvfeats, testBody_wvfeats = word2vec_features(testHeadLine, testBody)
print('Test word2vec features generated....')
testSentiment_feats = sentiment_features(testHeadLine, testBody)
print('Test sentiment features generated....')
testBase_feats = generate_features(testHeadLine, testBody)
print('Test baseline features generated....')
test_wvFeats = []
for x in range(len(testHead_wvfeats)):
    test_wvFeats.append(np.concatenate((testHead_wvfeats[x], testBody_wvfeats[x])))
    
test_X = np.hstack((test_wvFeats, testSentiment_feats))
test_X = np.hstack((test_X, testBase_feats))

/home/sanket/IIITH/Sem 4/Project/Project/output_data/word2vec.pickle
Test word2vec features generated....
Test sentiment features generated....
word overlap fets generated...
refuting fets generated...
polarity fets generated...
Test baseline features generated....


In [56]:
#Testing
test_pred = clf.predict_proba(test_X)
test_p = np.argmax(test_pred, axis=1)
testPredictions = [round(value) for value in test_p]

In [59]:
res = testPredictions[0]
print_label(res)

agree
